In [8]:
import os
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import numpy as np

In [9]:
def save_history(history, result_file):
    loss = history.history['loss']
    acc = history.history['acc']
    val_loss = history.history['val_loss']
    val_acc = history.history['val_acc']
    nb_epoch = len(acc)

    with open(result_file, "w") as fp:
        fp.write("epoch\tloss\tacc\tval_loss\tval_acc\n")
        for i in range(nb_epoch):
            fp.write("%d\t%f\t%f\t%f\t%f\n" % (i, loss[i], acc[i], val_loss[i], val_acc[i]))

In [10]:
classes = ['Tulip', 'Snowdrop', 'LilyValley', 'Bluebell', 'Crocus',
           'Iris', 'Tigerlily', 'Daffodil', 'Fritillary', 'Sunflower',
           'Daisy', 'ColtsFoot', 'Dandelion', 'Cowslip', 'Buttercup',
           'Windflower', 'Pansy']

batch_size = 32
nb_classes = len(classes)

img_rows, img_cols = 150, 150
channels = 3

train_data_dir = 'train_images'
validation_data_dir = 'test_images'

# 17カテゴリあり、各カテゴリに80種類ずつの画像がある 17*80=1360
# 1360からテスト用の画像 170を引いて、1190個
nb_train_samples = 1190
# 全カテゴリにおいて10個の画像をランダムに抽出
nb_val_samples = 170
nb_epoch = 9

steps_per_epoch =  int(nb_train_samples/batch_size)

In [11]:
result_dir = 'results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

In [12]:
# VGG16モデルと学習済み重みをロード
# Fully-connected層（FC）はいらないのでinclude_top=False）
input_tensor = Input(shape=(img_rows, img_cols, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

In [13]:
# 全結合層を構築
# Flattenへの入力指定はバッチ数を除く
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(nb_classes, activation='softmax'))

In [14]:
# VGG16と全結合層を接続
model = Model(inputs=vgg16.input, outputs=top_model(vgg16.output))

# 最後のconv層の直前までの層は、学習済のため、学習させない
for layer in model.layers[:15]:
    layer.trainable = False

optimizer_adam = optimizers.Adam(lr=1e-4)
optimizer_sgd = optimizers.SGD(lr=1e-4, momentum=0.9)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer_adam,
              metrics=['accuracy'])

# テスト用の画像の生成
http://aidiary.hatenablog.com/entry/20161212/1481549365

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

 # サブディレクトリをクラス名にしておくと、テスト用の画像を生成してくれる
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

Found 1190 images belonging to 17 classes.
Found 170 images belonging to 17 classes.


# 学習の実施

In [ ]:
# Fine-tuning
history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=nb_epoch,
    validation_data=validation_generator,
    validation_steps=nb_val_samples)

Epoch 1/9
37/37 [==============================] - 820s 22s/step - loss: 2.3709 - acc: 0.2628 - val_loss: 1.2021 - val_acc: 0.6652
Epoch 2/9
36/37 [============================>.] - ETA: 4s - loss: 1.1454 - acc: 0.6435

In [ ]:
model.save_weights(os.path.join(result_dir, 'finetuning.h5'))
save_history(history, os.path.join(result_dir, 'history_finetuning.txt'))